# Course Introduction

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Ongoing&color=orange)

<!-- Place this tag where you want the button to render. -->
<a class="github-button" href="https://github.com/particle1331/steepest-ascent" data-color-scheme="no-preference: dark; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star particle1331/steepest-ascent on GitHub">Star</a>
<!-- Place this tag in your head or just before your close body tag. -->
<script async defer src="https://buttons.github.io/buttons.js"></script> 


In this course, we will talk about best practices and tools for putting machine learning to production. The field for this is called **MLOps**. As an example, we will look at predicting the duration of a taxi ride (e.g. with a ride-hailing app). So what are best practices for putting ML models to production?

To answer that question, we first look at a simplified view of the process for machine learning projects. The first stage is **design**. We think about whether machine learning is the best solution for our problem. If so, we proceed to the next stage which is **train**. Here we train and experiment with different models, and after this stage we have a model which we can use for new data. Finally, we are in the **operate** stage. Suppose that the model is now in production, e.g. in a web service through an API, and then the customer can communicate with this API to provide an approximation of the ride duration given trip data. This process is called deployment. Part of the operate stage, is monitoring whether the model is performing well, or whether it is degrading or becoming worse. 

```{margin}
⚠️ **Attribution:** These are notes for [Week 1](https://www.youtube.com/watch?v=s0uaFZSzwfI&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK&index=3) of the [MLOps Zoomcamp](https://github.com/DataTalksClub/mlops-zoomcamp). This is a free course from [DataTalks.Club](https://github.com/DataTalksClub).
```

MLOps include tools and best practices for organizing experiments, and making sure we can reproduce the results of our experiments. This also includes tools and best practices that for retraining and deploying models, e.g. in one click, and monitor the quality of this model in production. This is a brief overview of the field of MLOps, we will talk about it in more detail in the next lessons.

## Environment preparation

In this section we create a server for creating a coding environment. In particular, a Linux machine which contains Docker, Python, and Jupyter. We also want a way to connect to this computer with VS Code, and perform port-forwarding, so we can use our local browser to access the remote Jupyter notebook server.

### Renting an EC2 instance

For this section, you will need an AWS account. You can [follow this tutorial](https://mlbookcamp.com/article/aws) for creating one. Find EC2 in the search bar, then you will be redirected to the **EC2 Dashboard**. Find and select "Launch Instance". Here we specify `mlops-zoomcamp` as the name of our virtual machine with Ubuntu with 64-bit (x86) architecture:


```{figure} ../../img/launch-instance-ec2.png
---
width: 45em
---
```

```{figure} ../../img/launch-instance-ec2-2.png
---
width: 45em
---

```

Next, select an instance type. Here a `t2.micro` is shown. But in the course, `t2.xlarge` was used. 

```{figure} ../../img/launch-instance-ec2-3.png
---
width: 40em
---
```

Choose any name for an RSA type key pair and save it with `.pem` format on the `.ssh/` folder of your local machine. This serves as credentials when connecting to the remote server. Select the created key pair (not shown here).

```{figure} ../../img/launch-instance-ec2-4.png
---
width: 35em
---
```

Skip network settings those are okay. Proceed to storage configuration where we set 30 GiB in storage. This much storage is needed for saving Docker images later in the course, and also for saving data.

```{figure} ../../img/launch-ec2-storage.png
---
width: 45em
---
```

After this, click "Launch instance" and your EC2 instance should successfully launch. We will now proceed to SSH to it to configure it. The dashboard should now look like as follows. Copy the instance IP address which is necessary for making an SSH connection to it. 

```{figure} ../../img/ec2-instances.png
---
width: 45em
---
```

```{figure} ../../img/ec2-instances-details.png
---
width: 45em
---
```

### SSH to the EC2 instance

In the terminal of your local machine execute the following two lines of code. The first line [`chmod 400`](https://chmodcommand.com/chmod-400/) means that the permission is set so that the user can read but can't write and execute the key pair file. The next line initiates the connection to the EC2 instance:

```bash
$ chmod 400 aws-m1.pem
$ ssh -i ~/.ssh/aws-m1.pem ubuntu@34.207.81.38

The authenticity of host '34.207.81.38 (34.207.81.38)' can't be established.
ED25519 key fingerprint is SHA256:YtGvgIhzmVvuu46GL4YnsX9anuYZYU/ZcnD+oI9/qaw.
This key is not known by any other names
Are you sure you want to continue connecting (yes/no/[fingerprint])? yes
Warning: Permanently added '34.207.81.38' (ED25519) to the list of known hosts.
Welcome to Ubuntu 22.04 LTS (GNU/Linux 5.15.0-1004-aws x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Sun May 22 17:28:46 UTC 2022

  System load:  0.36083984375     Processes:             106
  Usage of /:   5.0% of 28.90GB   Users logged in:       0
  Memory usage: 20%               IPv4 address for eth0: 172.31.24.183
  Swap usage:   0%

0 updates can be applied immediately.


The list of available updates is more than a week old.
To check for new updates run: sudo apt update


The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.

To run a command as administrator (user "root"), use "sudo <command>".
See "man sudo_root" for details.

ubuntu@ip-172-31-24-183:~$
```

To make it easier to connect to the remote machine, we can edit the `.ssh/config` file to include:

```
Host mlops-zoomcamp
    HostName 34.207.81.38
    User ubuntu
    IdentityFile ~/.ssh/aws-m1.pem
    StrictHostKeyChecking no
```

Then, we can simply run the following to SSH into our instance using:

```
$ ssh mlops-zoomcamp
```

### Installing Anaconda

We are now logged in. But there is nothing in this computer. First, we need to install Python.
Installation in our Ubuntu machine is easy, we only need to get download links which we can then install using `wget`. So we go to the [Anaconda downloads page](https://www.anaconda.com/products/distribution#Downloads) to get the link for the Linux 64-bit (x86) installer. The second step of this installation may take a while. The outputs are skipped for the sake of clarity:

```
ubuntu@ip-172-31-24-183:~$ wget https://repo.anaconda.com/archive/Anaconda3-2022.05-Linux-x86_64.sh
ubuntu@ip-172-31-24-183:~$ bash Anaconda3-2022.05-Linux-x86_64.sh
ubuntu@ip-172-31-24-183:~$ source ~/.bashrc
(base) ubuntu@ip-172-31-24-183:~$
```

The first command will save a bash script which is executed in the next command. This installs Anaconda after agreeing to the terms of use and confirming the install location. The `(base)` indicates that we are in the base `conda` environment; we can check for the Python installation that is being used:


```bash
(base) ubuntu@ip-172-31-24-183:~$ which python
/home/ubuntu/anaconda3/bin/python
```

### Installing Docker and Docker Compose

Running the following commands installs Docker. The last two is there so we can run `docker` without `sudo`. The outputs are skipped for clarity.

```bash
(base) ubuntu@ip-172-31-24-183:~$ sudo apt update
(base) ubuntu@ip-172-31-24-183:~$ sudo apt install docker.io
(base) ubuntu@ip-172-31-24-183:~$ sudo groupadd docker
(base) ubuntu@ip-172-31-24-183:~$ sudo usermod -aG docker $USER
```

Next create a `soft` directory where we will install Docker Compose. To get the download link for this, we go to the [releases page](https://github.com/docker/compose/releases) of Docker Compose in GitHub. There we will find the release for Linux x86 in the Assets section. Copy the link and perform the following install:

```bash
(base) ubuntu@ip-172-31-24-183:~$ mkdir soft
(base) ubuntu@ip-172-31-24-183:~$ cd soft
(base) ubuntu@ip-172-31-24-183:~/soft$ wget https://github.com/docker/compose/releases/download/v2.5.1/docker-compose-linux-x86_64 -O docker-compose
(base) ubuntu@ip-172-31-24-183:~/soft$ chmod +x docker-compose
(base) ubuntu@ip-172-31-24-183:~/soft$ ls
docker-compose
```

Here `-O` is to specify the save name, while [`chmod +x`](https://linuxtect.com/what-is-chmod-x-command-in-linux/) just makes the file executable. Next, we add the `soft` directory to the `PATH`. We can do this with `vim`:

```bash
vi ~/.bashrc
```

where we add a new line `export PATH="${HOME}/soft:${PATH}"` at the end of the file. To apply this change, we run the following command. The last output tells us that `soft` has been added to the path.

```bash
(base) ubuntu@ip-172-31-24-183:~/soft$ source ~/.bashrc
(base) ubuntu@ip-172-31-24-183:~/soft$ cd ..
(base) ubuntu@ip-172-31-24-183:~$ which docker-compose
/home/ubuntu/soft/docker-compose
```

Restart your VM instance. After restarting, we test if `docker` has been properly installed:

```bash
(base) ubuntu@ip-172-31-24-183:~$ docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
2db29710123e: Pull complete
Digest: sha256:80f31da1ac7b312ba29d65080fddf797dd76acfb870e677f390d5acba9741b17
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.
[...]

### Connecting with VS Code

Now that the coding environments are now setup, we can now put code into our remote computer. For read only access, we can use the HTTPS link to our fork of the [MLOps Zoomcamp code repository](https://github.com/DataTalksClub/mlops-zoomcamp). We use this to clone the repo into our remote instance:

```bash
(base) ubuntu@ip-172-31-24-183:~$ git clone https://github.com/particle1331/mlops-zoomcamp.git

Cloning into 'mlops-zoomcamp'...
remote: Enumerating objects: 351, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 351 (delta 68), reused 66 (delta 56), pack-reused 265
Receiving objects: 100% (351/351), 899.55 KiB | 17.64 MiB/s, done.
Resolving deltas: 100% (173/173), done.
```

We want to get access to the instance with VS code, so we have an IDE to work with. Open remote window by clicking the icon on lower left of the VS code window. Then click "Connect to Host". Then click `mlops-zoomcamp`. Note that this requires the [Remote - SSH](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-ssh) extension installed.

```{figure} ../../img/ec2-vscode-connected.png
---
width: 45em
---
```

Now, how about Jupyter notebooks? We want to connect to the remote Jupyter server with our local machine (i.e. using our browser). First, let's spin up a Jupyter notebook server in our remote instance:

```bash
(base) ubuntu@ip-172-31-24-183:~/mlops-zoomcamp$ jupyter notebook
[I 18:10:20.555 NotebookApp] Writing notebook server cookie secret to /home/ubuntu/.local/share/jupyter/runtime/notebook_cookie_secret
[I 2022-05-22 18:10:22.403 LabApp] JupyterLab extension loaded from /home/ubuntu/anaconda3/lib/python3.9/site-packages/jupyterlab
[I 2022-05-22 18:10:22.403 LabApp] JupyterLab application directory is /home/ubuntu/anaconda3/share/jupyter/lab
[I 18:10:22.409 NotebookApp] Serving notebooks from local directory: /home/ubuntu/mlops-zoomcamp
[I 18:10:22.409 NotebookApp] Jupyter Notebook 6.4.8 is running at:
[I 18:10:22.409 NotebookApp] http://localhost:8888/?token=db2b40b5ea1c3bc0f27d8838afa0c20debf7f99dd11e0ae7
[I 18:10:22.409 NotebookApp]  or http://127.0.0.1:8888/?token=db2b40b5ea1c3bc0f27d8838afa0c20debf7f99dd11e0ae7
[I 18:10:22.409 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[W 18:10:22.415 NotebookApp] No web browser found: could not locate runnable browser.
[C 18:10:22.416 NotebookApp]

    To access the notebook, open this file in a browser:
        file:///home/ubuntu/.local/share/jupyter/runtime/nbserver-6118-open.html
    Or copy and paste one of these URLs:
        http://localhost:8888/?token=db2b40b5ea1c3bc0f27d8838afa0c20debf7f99dd11e0ae7
     or http://127.0.0.1:8888/?token=db2b40b5ea1c3bc0f27d8838afa0c20debf7f99dd11e0ae7
```

Obviously, we can't connect to this server from our local machine using these URLs. To connect to this, we need port forwarding. We will use VS Code which gives an easy way for configuring this. Simply open the terminal in VS code and click the PORTS tab. Here you will see the remote port and its corresponding local address.

```{figure} ../../img/ec2-vscode-ports.png
---
width: 45em
---
```

Since the local address is `8889`, we put the following URL in our browser:

```
http://localhost:8889/?token=db2b40b5ea1c3bc0f27d8838afa0c20debf7f99dd11e0ae7
```

```{figure} ../../img/ec2-jupyter.png
---
width: 45em
---
```

## Training a ride duration prediction model


In this section, we look at how the model that will be deployed is developed and trained. 

```{figure} ../../img/ride-sharing-prediction.png
```

In [3]:
import pandas as pd

## Conclusion

In this session, we created an EC2 instance. 


Please star the code repository if you found this helpful! 

<!-- Place this tag where you want the button to render. -->
<a class="github-button" href="https://github.com/particle1331/steepest-ascent" data-color-scheme="no-preference: dark; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star particle1331/steepest-ascent on GitHub">Star</a>

<!-- Place this tag in your head or just before your close body tag. -->
<script async defer src="https://buttons.github.io/buttons.js"></script>
